<a href="https://colab.research.google.com/github/miczkejedrzej/MNLP-project-1/blob/main/Modelisation_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import of the training data

from google.colab import files
uploaded = files.upload()

# Import of moduls

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LeakyReLU
import matplotlib.pyplot as plt


from tqdm.auto import tqdm
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Loading the data
train_df = pd.read_json('train_df_complete.json', orient="records", lines=True)
dev_df = pd.read_json('dev_df_complete.json', orient="records", lines=True)

Saving dev_df_complete.json to dev_df_complete.json
Saving train_df_complete.json to train_df_complete.json


In [2]:
list_var = ['subclass_depth', 'subclasses', 'instances_of', 'instances_of_up', 'date', 'nb_lang',
       'descr_num_nouns', 'descr_num_verbs',
       'descr_num_adjectives', 'descr_has_location', 'descr_has_ethnic_group',
       'descr_has_event', 'type_cat', 'category_architecture',
       'category_biology', 'category_comics and anime', 'category_fashion',
       'category_films', 'category_food', 'category_geography',
       'category_literature', 'category_media', 'category_music',
       'category_politics', 'category_sports', 'category_transportation',
       'category_visual arts', 'category_combined',
       'main_country_cat'
]

X_train = train_df[list_var]
y_train = to_categorical(train_df['label_int'], num_classes = 3)

X_dev = dev_df[list_var]
y_dev = to_categorical(dev_df['label_int'], num_classes = 3)

In [6]:
X_train.head()

,subclass_depth,subclasses,instances_of,instances_of_up,date,nb_lang,descr_num_nouns,descr_num_verbs,descr_num_adjectives,descr_has_location,...,category_geography,category_literature,category_media,category_music,category_politics,category_sports,category_transportation,category_visual arts,category_combined,main_country_cat
0,1,0,0,1,1949,126.0,2,0,1,0,...,0,0,0,0,1,0,0,0,0,1
1,1,0,0,1,1592,72.0,4,0,1,0,...,0,0,0,0,1,0,0,0,0,1
2,1,0,0,1,165,83.0,1,0,1,0,...,0,0,0,0,1,0,0,0,0,1
3,1,0,0,1,1933,42.0,2,0,1,0,...,0,0,0,0,1,0,0,0,0,1
4,1,0,0,1,1913,174.0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,1


In [8]:
y_train

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       ...,
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]])

In [9]:
X_dev.head()

,subclass_depth,subclasses,instances_of,instances_of_up,date,nb_lang,descr_num_nouns,descr_num_verbs,descr_num_adjectives,descr_has_location,...,category_geography,category_literature,category_media,category_music,category_politics,category_sports,category_transportation,category_visual arts,category_combined,main_country_cat
0,1,0,0,3,1900,75,2,1,1,1,...,0,0,0,0,0,1,0,0,0,1
1,1,0,0,1,1957,10,2,0,0,1,...,0,0,0,1,0,0,0,0,0,1
2,1,0,0,2,1998,121,1,2,0,0,...,0,0,0,0,0,0,0,0,0,1
3,1,0,0,1,2011,28,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,1,0,0,1,1900,67,5,0,1,0,...,0,0,0,0,0,0,0,0,1,1


In [10]:
y_dev

array([[0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1

In [3]:
# Callbacks
early_stop = callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5)

# Model
model = models.Sequential()
model.add(layers.Input(shape=(29,)))

for units in [1024, 1024, 512, 512, 256, 256, 128, 128, 64, 64, 32, 32]:
    model.add(layers.Dense(units))
    model.add(LeakyReLU(alpha=0.1))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.3 if units > 128 else 0.2))

model.add(layers.Dense(3, activation='softmax'))

# Compilation
model.compile(optimizer=Adam(learning_rate=1e-3),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# Training
history = model.fit(X_train, y_train,
                    epochs=200,
                    batch_size=64,
                    callbacks=[early_stop, reduce_lr])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 1024)           │        30,720 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 1024)           │         4,096 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1024)           │     1,049,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_1 (LeakyReLU)       │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 1024)           │         4,096 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_2 (LeakyReLU)       │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_3 (LeakyReLU)       │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_4 (LeakyReLU)       │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_5 (LeakyReLU)       │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 2,146,083 (8.19 MB)

 Trainable params: 2,138,019 (8.16 MB)

 Non-trainable params: 8,064 (31.50 KB)

Epoch 1/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 29s 97ms/step - accuracy: 0.3815 - loss: nan - learning_rate: 0.0010
Epoch 2/200
22/98 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2858 - loss: nan

/usr/local/lib/python3.11/dist-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)
/usr/local/lib/python3.11/dist-packages/keras/src/callbacks/callback_list.py:145: UserWarning: Learning rate reduction is conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss,learning_rate.
  callback.on_epoch_end(epoch, logs)


98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.2936 - loss: nan - learning_rate: 0.0010
Epoch 3/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.2994 - loss: nan - learning_rate: 0.0010
Epoch 4/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.2975 - loss: nan - learning_rate: 0.0010
Epoch 5/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.2898 - loss: nan - learning_rate: 0.0010
Epoch 6/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.2914 - loss: nan - learning_rate: 0.0010
Epoch 7/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.3051 - loss: nan - learning_rate: 0.0010
Epoch 8/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.2950 - loss: nan - learning_rate: 0.0010
Epoch 9/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.2955 - loss: nan - learning_rate: 0.0010
Epoch 10/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.3096 - loss: nan - learning_rate: 0.0010
Epoch 11/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - acc

In [4]:
val_loss, val_accuracy = model.evaluate(X_dev, y_dev)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")

10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 83ms/step - accuracy: 0.3678 - loss: nan
Validation Loss: nan
Validation Accuracy: 0.38999998569488525


In [5]:
train_loss, train_accuracy = model.evaluate(X_train, y_train)
print(f"Train Loss: {train_loss}")
print(f"Train Accuracy: {train_accuracy}")

196/196 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.2864 - loss: nan
Validation Loss: nan
Validation Accuracy: 0.29931211471557617


# Random Forest

In [11]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_dev_scaled = scaler.transform(X_dev)

# Create random forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf_model.fit(X_train_scaled, y_train.argmax(axis=1))

# Prediction
y_dev_pred = rf_model.predict(X_dev_scaled)

# Compute accuracy
dev_accuracy = accuracy_score(y_dev.argmax(axis=1), y_dev_pred)
print(f"Accuracy on the development set: {accuracy * 100:.2f}%")


Accuracy on the development set: 74.00%


In [12]:
# Prediction on training set
y_train_pred = rf_model.predict(X_train_scaled)

# Compute training accuracy
train_accuracy = accuracy_score(y_train.argmax(axis=1), y_train_pred)
print(f"Accuracy on the training set: {train_accuracy * 100:.2f}%")

Accuracy on the training set: 99.70%
